# Exercise 15: Power Analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below. 

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill. 

`c` is reading comprehension, and `x` is word reading experience.

In [1]:
sample_size = 100 # How many children in data set? 
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0. 
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
        age = runif(sample_size, age_lo, age_hi)
        x = beta_x0 + beta_xa*age + rnorm(sample_size, mean=0, sd=sd_x)
        c = beta_c0 + beta_ca*age + beta_cx*x + rnorm(sample_size, mean= 0, sd=sd_c)
        return(data.frame(age=age,x=x,c=c)) # it's actually bad form to have a variable named "c" in R, my bad...
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,94.28135,-31.13855,5.581514
2,179.49142,133.65871,520.439172
3,189.51087,23.60079,270.354938
4,174.21059,44.55354,192.887508
5,127.77705,34.64880,294.244699
6,168.07952,88.38424,453.404238


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data. 

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously. 

In [2]:
# WRITE YOUR CODE HERE
library(mediation)

run_analysis = function(dat){
    lm1 = lm(x~age,data=dat)
    lm2 = lm(c~age+x,data=dat)
    med = mediate(lm1,lm2,treat="age",mediator="x")
    c(med$d0,med$d0.p,med$z0,med$z0.p)
}

print(run_analysis(dat))

Warning message:
"package 'mediation' was built under R version 4.2.3"
Loading required package: MASS

Loading required package: Matrix

Warning message:
"package 'Matrix' was built under R version 4.2.2"
Loading required package: mvtnorm

Loading required package: sandwich

Warning message:
"package 'sandwich' was built under R version 4.2.2"
mediation: Causal Mediation Analysis
Version: 4.5.0




[1] 1.311531 0.000000 1.246265 0.000000


---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [3]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)
    
    # Start simulating
    for (i in 1:num_simulations) {
        dat_temp = simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
        simouts[i,] = run_analysis(dat_temp)
    }
    
    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[,2] <= alpha)
    ADE_cov =  mean(simouts[,4] <= alpha)

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01. 

In [4]:
# WRITE YOUR CODE HERE
set.seed(1)
repeat_analysis(10, 0.01, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)


$ACME_cov
[1] 0.9

$ADE_cov
[1] 0.6

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run. 

In [5]:
# WRITE YOUR CODE HERE
df = data.frame(matrix(nrow = 0, ncol = 2))

for (i in c(50,75,100,125,150)){
    df = rbind(df, repeat_analysis(10, 0.01, i, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c))
    }

Print your results. 

In [6]:
# WRITE YOUR CODE HERE
print(df)

  ACME_cov ADE_cov
1      0.5     0.2
2      0.7     0.4
3      0.8     0.8
4      0.8     1.0
5      1.0     0.8


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above? 

> We probably want a sample size of at least 100 to get good power.

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> Age was accounted for in the simulated x-values so it would be more difficult to detect a direct effect since the simulation specifically included an indirect effect.

**DUE:** 5pm EST, April 5, 2023

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*